<a href="https://colab.research.google.com/github/mar7i4ka/Lin_Reg/blob/main/single_layer_fixed_lut_digits.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# ==========================================
# 1. Fixed-Point ⇄ Float Conversion Helpers
# ==========================================

TOTAL_BITS = 8     # total bits for signed fixed-point (including sign)
FRAC_BITS  = 4     # number of fractional bits (so Q3.4)

In [3]:
def float_to_fixed(x, total_bits=TOTAL_BITS, frac_bits=FRAC_BITS):
    """
    Convert a float x into an 8-bit signed fixed-point integer (Q3.4).
    """
    scale   = 1 << frac_bits                  # 2^frac_bits
    max_int = (1 << (total_bits - 1)) - 1     # +127 for 8-bit
    min_int = - (1 << (total_bits - 1))       # -128 for 8-bit
    xi      = int(np.round(x * scale))
    xi_clipped = max(min(xi, max_int), min_int)
    return np.int32(xi_clipped)

In [4]:
def fixed_to_float(x_fp, frac_bits=FRAC_BITS):
    """
    Convert an int32 in Q3.4 back to a Python float.
    """
    return float(x_fp) / (1 << frac_bits)

In [5]:
# ==========================================
# 2. Build 256-Entry Sigmoid LUT (Q3.4)
# ==========================================

LUT_SIZE = 256
LUT_MIN  = -6.0
LUT_MAX  = +6.0

# Sample 256 points in [LUT_MIN, LUT_MAX]
lut_x = np.linspace(LUT_MIN, LUT_MAX, LUT_SIZE)
# Compute float sigmoid at those points
lut_yf = 1.0 / (1.0 + np.exp(-lut_x))
# Quantize each sigmoid output to Q3.4
lut_fixed = np.array([float_to_fixed(v) for v in lut_yf], dtype=np.int32)

def sigmoid_lut(z_fp):
    """
    Given a signed fixed-point input z_fp (Q3.4),
    return sigmoid(z_fp) in Q3.4 by LUT lookup.
    """
    z_f = fixed_to_float(z_fp)
    idx = int(np.round((z_f - LUT_MIN) * (LUT_SIZE - 1) / (LUT_MAX - LUT_MIN)))
    idx = max(0, min(LUT_SIZE - 1, idx))
    return lut_fixed[idx]  # still Q3.4

In [6]:
# ==========================================
# 3. Load & Preprocess “digits” Dataset
# ==========================================

digits = load_digits()
X = digits.data / 16.0      # original pixels ∈ {0,…,16} → normalize to [0,1]
y = digits.target           # integers 0–9

# 80% train / 20% test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [8]:
# ==========================================
# 4. Train Float “Oracle” Single-Layer Model
# ==========================================
# This is exactly one matrix (10x64) + 10 biases (OvR logistic regression).
clf = LogisticRegression(
    solver='liblinear',
    multi_class='ovr',
    max_iter=1000,
    random_state=42
)
clf.fit(X_train, y_train)

# Extract float weights & biases:
#   Wf shape = (10, 64), bf shape = (10,)
Wf = clf.coef_.astype(np.float32)
bf = clf.intercept_.astype(np.float32)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(


In [9]:
# ==========================================
# 5. Quantize Weights & Biases to Q3.4
# ==========================================
Wq = np.vectorize(lambda v: float_to_fixed(v))(Wf)  # int32 → 10×64 (Q3.4)
bq = np.vectorize(lambda v: float_to_fixed(v))(bf)  # int32 → length 10


In [10]:
# 6. Define Fixed-Point Single-Layer Forward
# ==========================================
def forward_one_layer_fp(x_fp, W_fp, b_fp):
    """
    Compute one single-layer network in pure fixed-point + LUT:
      x_fp:  int32 array of length 64 (input in Q3.4)
      W_fp:  int32 array shape (10, 64) (fixed-point weights Q3.4)
      b_fp:  int32 array length 10 (biases Q3.4)
    Returns a length-10 float array (dequantized sigmoid outputs).
    """
    out_fp = np.zeros(10, dtype=np.int32)

    # For each of the 10 output neurons:
    for c in range(10):
        acc = np.int32(0)
        # Multiply-accumulate over 64 inputs (all Q3.4)
        for i in range(64):
            prod = np.int32(x_fp[i]) * np.int32(W_fp[c, i])  # Q7.8 intermediate
            acc  = np.int32(acc + (prod >> FRAC_BITS))        # shift back to Q3.4

        acc = np.int32(acc + b_fp[c])  # add bias (Q3.4)

        # Activation via LUT (still Q3.4)
        out_fp[c] = sigmoid_lut(acc)

    # Dequantize each output to float
    return np.array([fixed_to_float(v) for v in out_fp])


In [11]:
# ==========================================
# 7. Verification on Test Set
# ==========================================
correct = 0
N_test  = len(X_test)

for idx in range(N_test):
    x_f  = X_test[idx].astype(np.float32)
    # 7.1. Quantize the 64-D input vector to Q3.4
    x_fp = np.vectorize(lambda v: float_to_fixed(v))(x_f)

    # 7.2. Run fixed-point forward pass
    y_pred_fp = forward_one_layer_fp(x_fp, Wq, bq)  # length-10 float

    pred = int(np.argmax(y_pred_fp))  # pick the highest sigmoid
    if pred == y_test[idx]:
        correct += 1

accuracy = correct / N_test * 100
print(f"Single-Layer (Fixed-Point+LUT) Accuracy on “digits”: {accuracy:.2f}%")


Single-Layer (Fixed-Point+LUT) Accuracy on “digits”: 95.83%


In [12]:
# ==========================================
# 8. (Optional) Compare to Float Outputs on a Few Samples
# ==========================================
print("\nA few sample comparisons (float vs. fixed-point):")
for i in range(5):
    x_f = X_test[i].astype(np.float32)
    y_float = clf.predict_proba(x_f.reshape(1, -1))[0]  # float probabilities
    # Note: clfs.predict_proba applies softmax, but for OvR/logistic we can treat
    #       the raw sigmoid outputs (as implemented above) similarly. We'll just call
    #       forward_one_layer_fp and compare its 10-vector to the float logistic odds.
    x_fp    = np.vectorize(lambda v: float_to_fixed(v))(x_f)
    y_fixed = forward_one_layer_fp(x_fp, Wq, bq)

    print(f"Sample {i+1}: true={y_test[i]}")
    print(f"  float-proba  = {[round(p, 3) for p in y_float]}")
    print(f"  fixed-sigmoid = {[round(v, 3) for v in y_fixed]}")
    print(f"  argmax_f={int(np.argmax(y_float))}, argmax_fp={int(np.argmax(y_fixed))}\n")



A few sample comparisons (float vs. fixed-point):
Sample 1: true=6
  float-proba  = [np.float64(0.004), np.float64(0.002), np.float64(0.0), np.float64(0.0), np.float64(0.003), np.float64(0.004), np.float64(0.969), np.float64(0.001), np.float64(0.011), np.float64(0.005)]
  fixed-sigmoid = [np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(1.0), np.float64(0.0), np.float64(0.0), np.float64(0.0)]
  argmax_f=6, argmax_fp=6

Sample 2: true=9
  float-proba  = [np.float64(0.008), np.float64(0.0), np.float64(0.0), np.float64(0.007), np.float64(0.004), np.float64(0.127), np.float64(0.0), np.float64(0.001), np.float64(0.006), np.float64(0.848)]
  fixed-sigmoid = [np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.062), np.float64(0.0), np.float64(0.0), np.float64(0.0), np.float64(0.75)]
  argmax_f=9, argmax_fp=9

Sample 3: true=3
  float-proba  = [np.float64(0.0), np.float64(0.001), np.